<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#深度SVM网络" data-toc-modified-id="深度SVM网络-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>深度SVM网络</a></span><ul class="toc-item"><li><span><a href="#SVM" data-toc-modified-id="SVM-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>SVM</a></span></li><li><span><a href="#深度SVM的网络结构" data-toc-modified-id="深度SVM的网络结构-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>深度SVM的网络结构</a></span></li></ul></li><li><span><a href="#深度PCA网络" data-toc-modified-id="深度PCA网络-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>深度PCA网络</a></span></li><li><span><a href="#深度ADMM网络" data-toc-modified-id="深度ADMM网络-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>深度ADMM网络</a></span><ul class="toc-item"><li><span><a href="#模型" data-toc-modified-id="模型-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>模型</a></span></li></ul></li><li><span><a href="#极限学习机" data-toc-modified-id="极限学习机-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>极限学习机</a></span><ul class="toc-item"><li><span><a href="#模型" data-toc-modified-id="模型-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>模型</a></span></li><li><span><a href="#深度极限学习机" data-toc-modified-id="深度极限学习机-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>深度极限学习机</a></span><ul class="toc-item"><li><span><a href="#模型" data-toc-modified-id="模型-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>模型</a></span></li></ul></li></ul></li><li><span><a href="#深度多尺度几何网络" data-toc-modified-id="深度多尺度几何网络-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>深度多尺度几何网络</a></span><ul class="toc-item"><li><span><a href="#深度脊波网络" data-toc-modified-id="深度脊波网络-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>深度脊波网络</a></span><ul class="toc-item"><li><span><a href="#模型" data-toc-modified-id="模型-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>模型</a></span></li></ul></li></ul></li><li><span><a href="#深度轮廓波网络" data-toc-modified-id="深度轮廓波网络-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>深度轮廓波网络</a></span><ul class="toc-item"><li><span><a href="#非下采样轮廓波" data-toc-modified-id="非下采样轮廓波-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>非下采样轮廓波</a></span></li><li><span><a href="#深度轮廓波网络" data-toc-modified-id="深度轮廓波网络-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>深度轮廓波网络</a></span></li></ul></li><li><span><a href="#深度森林" data-toc-modified-id="深度森林-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>深度森林</a></span></li></ul></div>

经典神经网络的缺点:可行域包含大量的局部极值点和鞍点;模型的训练性能和泛化性能严重依赖数据量且容易出现过拟合,并且网络的设计(层数\隐单元个数\和非线性函数\参数初始化\学习率)包含很多人为给定的超参数,虽然可以利用格式搜索的方法确定超参数,但计算代价太大.,<br/>

深度融合网络的优点:凸优化目标函数;自动确定模型规模;有效预防过拟合现象.
## 深度SVM网络
### SVM
如果数据集是线性可分的,则模型可表示为:$f(\boldsymbol{x}=\boldsymbol{x^Tx}+b$,对应的优化目标和约束条件为:$$\underset{\boldsymbol{x},b}{min}\quad \frac{1}{2}||\boldsymbol{w}||^2$$ $$s.t.\quad y_i(\boldsymbol{w}^T\boldsymbol{x}_i+b)\ge 1,\quad i=1,2,...,N$$
其无约束形式为$$\underset{\boldsymbol{\alpha}}{max}\underset{\boldsymbol{w},b}{min}L(\boldsymbol{w},b,\boldsymbol{\alpha})=\frac{1}{2}||\boldsymbol{w}||^2+\sum_i{\alpha_i(1-y_i(\boldsymbol{w}^T\boldsymbol{x}_i+b))}$$
对应$\boldsymbol{w}$解为$\boldsymbol{w}=\sum_i{\alpha_i\cdot y_i\cdot \boldsymbol{x}_i}$<br/>
若数据集线性不可分,则可通过特征学习(如升维)将其变为线性可分,对应约束条件变为:$$s.t.\quad y_i(\boldsymbol{w}^T\phi(\boldsymbol{x}_i)+b)\ge 1$$
对应$\boldsymbol{w}$解为$\boldsymbol{w}=\sum_i{\alpha_i\cdot y_i\cdot \phi(\boldsymbol{x}_i})$<br/>
模型因而简记为:$$\boldsymbol{y}=\sum_{i=1}^N{\alpha_i\cdot\kappa(\boldsymbol{x},\boldsymbol{x}^T)}$$

### 深度SVM的网络结构
深度SVM网络的结构核心在于非线性单元的设计.<br/>
模型结构为:$$\begin{cases}\boldsymbol{h}=(h_1,h_2,....,h_K)\\h_k=\sum_{n=1}^N{\alpha_k^{(n)}\kappa(\boldsymbol{x}^{(n)},\boldsymbol{x})+b_k}\end{cases}$$
其中,$\kappa$为核函数,SVM**隐层每个节点的输出为$h_k$**,$\alpha_k^{(n)},b_k$为第k个待学习的参数与偏置.$h_k$并不是一维的.接下来将隐层特征$\boldsymbol{h}$作为下一个非线性单元的输入,即有:$$\begin{cases}\boldsymbol{y}=\sum_{n=1}^N{\beta^{(n)}\cdot \kappa(\boldsymbol{h}^{(n)},\boldsymbol{h})+c}\\\boldsymbol{h}^{(n)}=[h_1^{(n)},h_2^{(n)},...,h_K^{(n)}]\\\boldsymbol{h}=[h_1,...,h_K]\end{cases}$$
其中,$\boldsymbol{h}^{(n)}$为输入$\boldsymbol{x}^{(n)}$*(第n个样本**到底是啥**)的隐层特征*.<br/>
其优化目标为:$$\underset{\theta}{min}J{\theta}=\frac{1}{N}\sum_{n=1}^N{||\hat{\boldsymbol{y}}^{(n)}-\boldsymbol{y}^{(n)}||^2_2}+\lambda\cdot R(\theta)$$
求解可以通过梯度下降方法.上述仅包含一个隐层的SVM网络不应称为深度SVM模型. 

通常深度SVM网络选用的的激活函数为**径向基函数(在网络中体现在哪一步?)**以解决SVM的两个问题:一是模型的性能取决于先验选择的核函数(核函数的选择?);二是具有单层可调整的参数的模型表征能力有限(不是跟第一条一样?).
SVM网络的优点在于:可以有效地防止过拟合,可以根据**支撑向量的个数自动确定模型的规模(?)**

## 深度PCA网络
深度学习的核心在于自适应(层级)特征的提取,而深度PCA网络是一种基于深度卷积神经网络架构的**PCA参数初始化**的无监督学习方法,其核心仍在于卷积流模块中卷积操作所使用的**滤波器的学习**.该网络的模式仍为(无监督)预训练+(有监督)精调的半监督学习方式.**(深度SVM网络可以采用这种方式吗?)**<br/>
其核心操作为:<br/>1)将n个$h\times h\times c$(卷积核尺寸)卷积核(**仍需初始参数?**)经卷积操作获得的$(\frac{w-h}{p}+1)^2$个卷积块,<br/>
2)将每一个卷积块先按行再按通道拉成一列,得到一个$(\frac{w-h}{p}+1)^2\times(h^2c)$的矩阵,<br/>
3)对矩阵进行奇异值分解,将奇异值按大到小排序,取前n个特征值对应的特征向量,获得一个$(h^2c)\times n$的矩阵,<br/>
4)将其按通道再按行'逆向'得到滤波器组即$h\times h\times h\times c@n$这个即可作为输入层至第一隐层的滤波器的初始参数.<br/>
5)进一步利用池化后的第二个隐层作为输入,重复上述步骤得到所有隐层间滤波器的初始参数.**(可以此作为初始参数的理论依据?)**

## 深度ADMM网络
深度学习问题中,超参数(层数,隐层结点,激活函数)的确定一直是难以进行量化分析的部分,而深度ADMM网络通过结合ADMM算法和深度学习实现该问题的量化分析.ADMM是一种凸优化问题求解算法，**通过引入中间变量将正则项中参量从单目标函数中脱离处理,通过两个子问题(基于损失项的凸优化问题与基于正则项的软阈值求解)的交替求解来逼近原目标的解**。ADMM网络类似于深度反卷积神经网络，使用ADMM算法优化每一个模块（即卷积稀疏编码下的合成滤波器与系数）。**其属于无监督学习**<br/>
### 模型
$$\begin{cases}\boldsymbol{y}=\boldsymbol{\Phi\cdot x}\\\boldsymbol{\alpha}=\boldsymbol{D\cdot x}\end{cases}$$
其中$\boldsymbol{y}$为观测数据,$\boldsymbol{x}$为原始场景,$\boldsymbol{\Phi}$为观测矩阵(一般观测特征小于原始特征),$\boldsymbol{\alpha}$为原始场景的某种先验,如稀疏性,$\boldsymbol{D}$为(稀疏性)滤波器(维度大于原始特征维度).<br/>
其优化目标为:$$\underset{\{x^{(n)}\},\boldsymbol{\Phi,D}}{min}J(\{x^{(n)}\},\boldsymbol{\Phi,D})=\\\frac{1}{2\cdot N}\sum_{n=1}^N{\left\{||\boldsymbol{y}^{(n)}-\boldsymbol{\Phi}\cdot\boldsymbol{x}^{(n)}||^2_2+\sum_{l=1}^L{\lambda_l\cdot\rho(\boldsymbol{D}_l\cdot\boldsymbol{x}^{(n)}}\right\}}$$
其中,$\rho(\cdot)$为某种正则函数,如关于稀疏性的$L_p$范数.

引入中间变量$\boldsymbol{z}_l=\boldsymbol{D}_l\cdot\boldsymbol{x}$,则原优化问题转化为无约束优化:
$$\underset{\boldsymbol{z},\{\boldsymbol{z}_l,\boldsymbol{\beta}\}}{min}\frac{1}{2}||\boldsymbol{y}-\boldsymbol{\Phi\cdot x}||_2^2+\sum_{l=1}^{L}{\lambda_l\cdot\rho(\boldsymbol{z}_l}-\sum_{l=1}^{L}{<\boldsymbol{\beta}_l,\boldsymbol{z}_l-\boldsymbol{D}_l\cdot\boldsymbol{x}>}+\sum_{l=1}^L{\frac{\gamma_l}{2}||\boldsymbol{z}_l-\boldsymbol{D}_l\cdot\boldsymbol{x}||_2^2}$$
由此可求得其参数迭代公式为:$$\begin{cases}
\boldsymbol{x}^{(t)}=\left[\boldsymbol{\Phi}^T\boldsymbol{\Phi}-\sum_{l=1}^L{\gamma_l\cdot(\boldsymbol{D}_l)^T\boldsymbol{D}_l}\right]^{-1}\left[\boldsymbol{\Phi}^T\cdot\boldsymbol{y}-\sum_{l=1}^L{(\boldsymbol{D}_l)^T\cdot(\boldsymbol{\beta}_l^{(t-1)}+\gamma_l\cdot\boldsymbol{z}_l^{(t-1)})}\right]\\
\boldsymbol{z}^{(t)}:\boldsymbol{z}^{(t)}_l=S\left(\boldsymbol{D}_l\cdot\boldsymbol{x}^{(t)}+\boldsymbol{\beta}_l^{(t-1)};\frac{\lambda_l}{\gamma_l}\right),l=1,2..L\\
\boldsymbol{\beta}^{(t)}:\boldsymbol{\beta}^{(t)}=\boldsymbol{\beta}^{(t-1)}_l+\eta\cdot(\boldsymbol{z}_l^{(t)}-\boldsymbol{D}_l\cdot\boldsymbol{x}^{(t)})\end{cases}$$

类比卷积神经网络,卷积层为线性处理后得到的若干特征图,记为模块$X^{(t)}$,即$\boldsymbol{x}^{(t)}$的更新,本质上是线性操作.其次非线性处理(激活函数),记为模块$C^{(t)}$,其中收缩函数$S(\cdot)$为非线性函数,即模块$X^{(t)}$到$Z^{(t)}$的非线性映射.最后参数更新层,即模块$M^{(t)}$包括$\boldsymbol{\beta}^{(t)}$,学习率$\eta$等.网络结构中的深度是指更新的迭代次数.

## 极限学习机
极限学习机是一种新型快速学习算法,对于单隐层神经网络,它可以随机初始化输入层与隐层之间的权值连接矩阵和偏置,并通过学习或训练得到隐层与输入层之间的权值矩阵.它的优势在于:只需要设置网络的隐层节点数,在算法执行过程中**不需要调整网络的输入层到隐层的权值及偏置**,并产生唯一解,具有学习速度快且泛化性能好的优点.**其属于有监督学习**.
### 模型
$$\begin{cases}\boldsymbol{h}_x=\sigma(\boldsymbol{W}_x\cdot\boldsymbol{x}+\boldsymbol{b}_x)\in\mathbb{R}^r\\
\boldsymbol{y}=\boldsymbol{\beta}\cdot\boldsymbol{h}_x+\boldsymbol{c}\in\mathbb{R}^s\end{cases}$$
**对于不同的输入,随机化权值与偏置$\boldsymbol{W,b}$也发生变化**.同时权值与偏置$\boldsymbol{W,b}$不需要学习,服从某种分布下的随机化获取.待学习的参数为$\boldsymbol{\beta,c}$,隐层节点个数为r,$\sigma(\cdot)$为激活函数.
其目标函数为$$\underset{\boldsymbol{\beta,c}}{min}\frac{1}{2}\sum_{n=1}^N{||\boldsymbol{y}^{(n)}-\hat{\boldsymbol{y}}^{(n)}||^2_2}$$


类似于最小二乘法,根据目标问题可以得到$\boldsymbol{\beta\cdot H=Y-C}$,令$\boldsymbol{C}$,利用Moore-Penrose广义逆,得到:
$$\boldsymbol{\beta=Y\cdot H^T(H\cdot H^T)^{-1}=Y\cdot H^{\dagger}}$$
已经证明,**一旦输入到隐层之间的权值矩阵和隐层偏置被随机确定,则隐层的输入矩阵$\boldsymbol{H}$被唯一确定,从而得到$\boldsymbol{\beta}$的唯一解**.其中,如果$\boldsymbol{H\cdot H}^T\in\mathbb{R}^{r\times r}$不可逆,则需加正则项从而$\boldsymbol{H}^{\dagger}=\boldsymbol{H}^T\left(\boldsymbol{H\cdot H}+\frac{1}{\lambda}\cdot \boldsymbol{I}_r\right)^{-1}$<br/>
其次,核极限学习机,是指跳过随机初始化权值矩阵与偏置,根据输入得到隐层输出的过程.隐层节点的个数大于样本的个数 **(?)**,为此引入核函数,使得
$$\boldsymbol{H^T\cdot H}=\left[\kappa(\boldsymbol{x}_i,\boldsymbol{x}_j)\right]_{N\times N}$$
进而$$\boldsymbol{H}^T\cdot \boldsymbol{h}_x=\begin{bmatrix}\kappa(\boldsymbol{x}_1,\boldsymbol{x})\\\vdots\\\kappa(\boldsymbol{x}_N,x)\end{bmatrix}$$

从而$$\boldsymbol{y}=\boldsymbol{Y}\cdot
\begin{bmatrix}\kappa(\boldsymbol{x}_1,\boldsymbol{x}_1) & \cdots & \kappa(\boldsymbol{x}_1,\boldsymbol{x}_N)\\ 
\vdots & \ddots & \vdots \\ \kappa(\boldsymbol{x}_N,\boldsymbol{x}_1) & \cdots & \kappa(\boldsymbol{x}_N,\boldsymbol{x}_N)\end{bmatrix}^{-1}\cdot 
\begin{bmatrix}\kappa(\boldsymbol{x}_1,\boldsymbol{x})\\ \vdots \\ \kappa(\boldsymbol{x}_N,\boldsymbol{x})\end{bmatrix}$$

### 深度极限学习机
深度极限学习机是基于极限学习机的**自编码网络**.其核心如上所述是学习参数$\beta$,参数初始化过程(测试数据)的期望输出与输入一致,因此是一种无监督学习,而精调阶段(训练数据)进行回归或分类器的设计.<br/>
#### 模型
$$\begin{cases}\boldsymbol{h}_l=\sigma(\boldsymbol{\beta}^{(l)}\cdot\boldsymbol{h}_{l-1}),l=1,2,...,L\\\boldsymbol{y}=\boldsymbol{\beta}_C\cdot\boldsymbol{h}_L\end{cases}$$
其中$\boldsymbol{\beta}^{(l)}$为无监督的方式下逐层训练的参数,$\boldsymbol{\beta}_C$为最后回归器或分类器设计阶段的参数.<br/>


其优化目标函数:$$\underset{\{\boldsymbol{\beta}^{(l)}\},\boldsymbol{\beta}_C}{min}\frac{1}{N}\sum_{n=1}^N{||\hat{\boldsymbol{y}}^{(n)}-\boldsymbol{y}^{(n)}||^2_2}+\lambda \sum_{l=1}^L{||\boldsymbol{\beta}^{(l)}||^2_F}+\lambda||\boldsymbol{\beta}_c||^2_F$$
参数初始化过程的采用如下的目标函数:
$$\underset{\boldsymbol{\beta}^{(l)}}{min}\frac{1}{T}\sum_{t=1}^T{||\boldsymbol{h}_t^{(l)}-\hat{\boldsymbol{h}}_t^{(l)}||^2_2}+\lambda\cdot||\boldsymbol{\beta}^{(l)}||^2_F$$
深度极限学习机可以提取出数据中高层次的抽象信息;极限学习机的理论避免了网络权值的反复迭代调整,提高了计算效率,半监督的逐层训练机制解决了很多实际问题中训练标签难以获取的问题.

## 深度多尺度几何网络
深度多尺度几何网络是指基于三代小波(如脊波,曲波,轮廓波,楔形波)所构建的深度神经网络,与传统的基于一代小波(Meyer小波,Morlet小波,Haar小波,Gaussian小波,Doubechies小波)所构建的神经网络不同之处在于:一是几乎不再使用(三代)小波作为激活函数来调整隐层线性输出后的扭曲程度或响应特性,而是将其更多地使用在权值矩阵初始化、多尺度或多通路网络的设计中,以避免过早地陷入局部最优，以及获取同一层级不同分辨率下的特征来提升网络表征能力。二是三代小波相比一代小波更容易获取输入场景多频带、多角度下的拓扑特性描述，更符合生物的视觉系统特性。
### 深度脊波网络
若$\phi(x)\in L^2(\mathbb{R})$满足容许性条件，即$$C_{\phi}=\int_{\mathbb{R}}\left(\frac{|\hat{\phi}(\omega)|^2}{|\omega|^d}\right)\,d\omega<+\infty$$其中，$$\hat{\phi}(\omega)=\int_{\mathbb{R}}\phi(x)e^{-j\omega x}\,dx$$称$\phi$为容许性神经激活函数。

进一步，由该函数产生的脊函数即脊波：$$\begin{cases}\phi_{(a,u,b)}(x)=\frac{1}{\sqrt{a}}\phi\left(\frac{u\cdot x-b}{a}\right)\\a\in\mathbb{R}^+,b\in\mathbb{R},u\in S^{d-1},||u||=1\end{cases}$$其中，$S^{d-1}$为单位球，$(a,u,b)$分别为尺度因子，平移因子和方向因子。<br/>通常将用脊波作为激活函数即脊波神经网络。


#### 模型
$$\begin{cases}\phi\left(\cfrac{\boldsymbol{u\cdot x-b}}{\boldsymbol{a}}\right)=\left(\phi\left(\cfrac{\boldsymbol{u}(1,:)\cdot\boldsymbol{x}-b(1)}{a(1)}\right),...,\phi\left(\cfrac{\boldsymbol{u}(s,:)\cdot\boldsymbol{x}-b(s)}{a(s)}\right)\right)^T\in\mathbb{R}^s\\
\boldsymbol{h}=\sigma(\boldsymbol{u}\cdot x-b)=\phi\left(\cfrac{\boldsymbol{u\cdot x-b}}{\boldsymbol{a}}\right)\\
\boldsymbol{y}=\boldsymbol{\omega\cdot h+\beta}\in\mathbb{R}^m\end{cases}$$
优化目标为:$$\underset{\theta}{min}J(\theta)=\cfrac{1}{2T}\sum_{t=1}^T{||\hat{\boldsymbol{y}}^{(t)}-\boldsymbol{y}^{(t)}||^2_2}+\cfrac{\lambda}{2}\cdot||\boldsymbol{u}||^2_F+\cfrac{\gamma}{2}||\boldsymbol{\omega}||_F^2$$
根据三层的脊波神经网络可以利用自编码的方式通过逐层堆栈的方式形成深度脊波网络,该模型的优点在于:半监督的学习方式;融合脊波的特性,即灵活的结构和快速并行的处理速度,及较强的容错性和鲁棒性.

## 深度轮廓波网络
轮廓波变化具有稀疏性、多尺度特性、多方向性、局部化、低冗余度、平移不变性、容易实现和计算高效等众多特性。轮廓波变换的核心思想是在多尺度的基础上实现方向信息的获取，其变换所对应的滤波器分为拉普拉斯塔式分解和方向滤波器组两个部分，其中拉普拉斯塔式滤波器主要完成奇异点的分离任务；方向滤波器主要完成奇异点的收集工作，即利用方向基本相同的准则，将奇异点收集到一个基函数上描述，常用的轮廓波变换有非下采样轮廓波变换、全相位轮廓波变换、基于小波的轮廓波变换、抗混叠轮廓波变换以及复轮廓波变换。
### 非下采样轮廓波
### 深度轮廓波网络

## 深度森林